In [2]:
import pandas as pd
import numpy as np
from itertools import product

# Substitua 'retail_store_inventory_opendata.csv' pelo caminho real do seu arquivo original
file_path = "./retail_store_inventory_opendata.csv"
df_original = pd.read_csv(file_path)

# 1. Estrutura e Distribuição
categorical_cols = ['Category', 'Region', 'Weather Condition', 'Holiday/Promotion', 'Seasonality']
numeric_cols = [
    'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 
    'Price', 'Discount', 'Competitor Pricing'
]

# 2. Novas Lojas e Produtos
num_new_stores = 15
num_new_products = 75
store_ids = [f'S{i:03d}' for i in range(1, num_new_stores + 1)]
product_ids = [f'P{i:04d}' for i in range(1, num_new_products + 1)]

# Obter datas únicas
dates = df_original['Date'].unique()

# 3. Criar o DataFrame Expandido
# Criar todas as combinações (Date, Store ID, Product ID)
new_index = pd.DataFrame(list(product(dates, store_ids, product_ids)), 
                         columns=['Date', 'Store ID', 'Product ID'])

# Amostragem das colunas categóricas
for col in ['Category', 'Region']:
    counts = df_original[col].value_counts(normalize=True)
    new_index[col] = np.random.choice(counts.index, size=len(new_index), p=counts.values)

# Mapear variáveis dependentes da Data
date_mapping = df_original.groupby('Date').agg({
    'Weather Condition': lambda x: x.mode()[0], 
    'Holiday/Promotion': lambda x: x.mode()[0],
    'Seasonality': lambda x: x.mode()[0]
}).reset_index()

df_expanded = new_index.merge(date_mapping, on='Date', how='left')

# Função de Geração de Dados (replicada)
def generate_similar_data(original_data, size, zero_ratio=0.02):
    sampled = np.random.choice(original_data, size=size, replace=True)
    std_dev = original_data.std()
    noise = np.random.normal(0, std_dev * 0.1, size=size)
    generated = sampled + noise
    generated[generated < 0] = 0
    
    # Zeros controlados
    if 'Units Sold' in original_data.name or 'Discount' in original_data.name:
        zero_indices = np.random.choice(size, size=int(size * zero_ratio), replace=False)
        generated[zero_indices] = 0
        
        if 'Units Sold' in original_data.name:
            generated = np.round(generated).astype(int)

    if 'Discount' in original_data.name:
        generated = np.clip(generated, 0, 50).astype(int)
        
    if 'Holiday/Promotion' in original_data.name:
        generated = np.random.randint(0, 2, size=size)
        
    if original_data.name in ['Inventory Level', 'Units Ordered', 'Demand Forecast', 'Competitor Pricing']:
        generated = np.round(generated, 2)
        if original_data.name in ['Inventory Level', 'Units Ordered']:
             generated = generated.astype(int)

    if 'Price' in original_data.name:
        generated = np.round(generated, 2)
        
    return generated

# Aplicar a função de geração a todas as colunas numéricas
for col in numeric_cols:
    df_expanded[col] = generate_similar_data(df_original[col], len(df_expanded))

# Organizar e salvar
final_cols = [
    'Date', 'Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 
    'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 
    'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing', 'Seasonality'
]

df_expanded = df_expanded[final_cols]
new_file_path = "./retail_store_inventory_expanded.csv"
df_expanded.to_csv(new_file_path, index=False)
print(f"O arquivo expandido foi salvo como '{new_file_path}' em sua máquina.")

O arquivo expandido foi salvo como './retail_store_inventory_expanded.csv' em sua máquina.
